In [1]:
import unicodedata
import re

import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
def remove_punct(string):
    nfkd = unicodedata.normalize('NFKD', string)
    return u"".join([c for c in nfkd if not unicodedata.combining(c)])

In [3]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
df_senate = pd.read_csv("clean_senate.xz",compression='xz',parse_dates=['date'],date_parser=dateparse,low_memory=False,index_col=0)

ValueError: 'date' is not in list

In [ ]:
df_senate["congressperson_name"] = df_senate["congressperson_name"].apply(remove_punct)

In [ ]:
def legislation_54_55(path):
    congress_man = {}
    with open(path,"r") as f:
        for line in f:
            politician = line.split(" ")
            start, _, end = politician[-3:]
            start = datetime.strptime(start,"%Y")
            end = datetime.strptime(end.strip(),"%Y")
            party, uf = politician[-5:-3]
            name = remove_punct(" ".join([i for i in politician[:-5] if i != ""]).upper())
            congress_man[name] = {"party":party,"uf":uf, "start": start,"end":end}
    return congress_man

In [ ]:
def legislation_55():
    congress_man = {}
    with open("2015_2019.txt","r") as f:
        for line in f:
            politician = line.split("/")[0].split(" ")[:-3]
            try:
                start, _, end = politician[-3:]
                party, uf = politician[-5:-3]
                start = datetime.strptime(start,"%Y")
                end = datetime.strptime(end,"%Y")
                
                name = remove_punct(" ".join([i for i in politician[:-5] if i != ""]).upper())
                congress_man[name] = {"party":party,"uf":uf, "start": start,"end":end}
            except Exception as e:
                pass
    return congress_man

In [ ]:
congress_man1 = legislation_54_55("2003_2011.txt")
congress_man2 = legislation_54_55("2009_2015.txt")
congress_man3 = legislation_55()

In [ ]:
def check(row, congress_man):
    if row["congressperson_name"] in congress_man and\
        row["date"] <= congress_man[row["congressperson_name"]]["end"] and\
        row["date"] >= congress_man[row["congressperson_name"]]["start"]:
            return congress_man[row["congressperson_name"]]["party"]
    return None

In [ ]:
def findparty(row):
    c1 = check(row, congress_man1)
    if c1: return c1
    else:
        c1 = check(row, congress_man2)
        if c1: return c1
        return check(row, congress_man3)

In [ ]:
len(df_senate)

In [ ]:
parties = df_senate.apply(findparty,axis=1)

In [ ]:
len(parties.isnull())

In [ ]:
# df_senate["party"] = parties

In [ ]:
# df_senate.head()

In [ ]:
# df_senate.to_csv(path_or_buf='clean_senate.xz', sep=';', compression='xz',chunksize=10000)